# Decision Tree Classification

In [1]:
import pandas as pd
import numpy as np
import random
from collections import Counter

### Importing the data
- The data can be found here: https://archive.ics.uci.edu/ml/datasets/Wine+Quality. The target variable will be 'color' and there are 12 features. 

In [2]:
red=pd.read_csv('winequality-red.csv',sep=';')
white=pd.read_csv('winequality-white.csv',sep=';')
white['color']=0
red['color']=1
red_index=random.sample(range(0,len(red)),100)         # taking a small sample so training is faster
white_index=random.sample(range(0,len(white)),100)
red_sample=red.iloc[red_index]
white_sample=white.iloc[white_index]
data=pd.concat([white_sample,red_sample],ignore_index=True)

### Splitting into training and test sets

In [7]:
target='color'
features=data.columns.drop(labels='color')
i_test=random.sample(range(0,200),50)
i_train=[]
for i in data.index:
    if i not in i_test:
        i_train.append(i)
train=np.array(data.loc[i_train])
X_train=np.array(data.loc[i_train,features])
y_train=np.array(data.loc[i_train,target])
X_test=np.array(data.loc[i_test,features])
y_test=np.array(data.loc[i_test,target])
test=np.array(data.loc[i_test])

In [8]:
X_test.shape

(50, 12)

### Needed functions

In [9]:
def unique_vals(rows, col):                # Gets the unique values for each colomn 
    return set([row[col] for row in rows])

def class_count(data):                    # counts the number of target variables
    counts = {}
    for row in data:
        label=row[-1]
        if label not in counts:
            counts[label] =0
        counts[label] +=1
    return counts
 
class Evaluator:                           # boolean values by columns with a predetermined by
    def __init__(self, column, value):
        self.column=column
        self.value=value
    def evaluate(self, example):
        val=example[self.column]
        return val >= self.value
    
def partition(rows, evaluator):            #boolean values evaluator class go into different lists
    true_rows, false_rows = [], []
    for row in rows:
        if evaluator.evaluate(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

### Gini impurity
$=1-\sum\limits_{i=1}^{J}pi^{2}$  
- J = is the number of target variables
- measures the probability of a value not being select. A value of zero means that the node has only one value
- source: https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity

In [10]:
def gini(rows):
    counts=class_count(rows)
    impurity=1
    for i in counts:
        prob_i=counts[i]/float(len(rows))
        impurity -= prob_i**2
    return impurity

### Information Gain
- parent = current uncertainty
- w = weighted impurity for children
$information_gain = parent - w*impurity(left) - (1-w)*impurity(right)$

In [11]:
def info_gain(left, right, current_uncertainty ):
    w=float(len(left))/len(left)+len(right)
    return current_uncertainty  - w*gini(left)-(1-w)*gini(right)

In [12]:
def find_best_split(rows):  # goes over all data to find the highest info gain
    best_gain=0
    best_evaluator=None
    current_uncertainty = gini(rows)
    for col in range(12):
        values=set([row[col] for row in rows])
        for val in values:
            evaluation=Evaluator(col, val)
            true_rows, false_rows = partition(rows, evaluation)
            
            if len(true_rows)==0 or len(false_rows)==0:
                continue
            gain = info_gain(true_rows, false_rows, current_uncertainty)
            
            if gain > best_gain:
                best_gain, best_evaluator = gain, evaluation
    return best_gain, best_evaluator

class Leaf:                    # counts the number of target values 
    def __init__(self, rows):
        self.predictions = class_count(rows)
class Decision_Node:
    def __init__(self, evaluator, true_branch, false_branch):   # holds reference to evaluation and children nodes
        self.evaluator=evaluator
        self.true_branch=true_branch
        self.false_branch=false_branch


In [13]:
def build_tree(rows):
    gain, evaluator=find_best_split(rows)
    if gain==0:
        return Leaf(rows)
    true_rows, false_rows = partition(rows, evaluator)
    true_branch=build_tree(true_rows)
    false_branch=build_tree(false_rows)
    return Decision_Node(evaluator, true_branch, false_branch)

In [14]:
def classify(row, node):
    if isinstance(node, Leaf):
        return node.predictions
    if node.evaluator.evaluate(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [15]:
tree=build_tree(train)
results=[]
for row in X_test:
    pred=classify(row, tree)
    for k,v in pred.items():
        results.append(k)
results=np.array(results)

### Comparison with Sklearn

In [33]:
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

In [46]:
clf = DecisionTreeClassifier()
clf.fit(X_train,y_train)
sk_predictions=clf.predict(X_test)

In [47]:
my_confusion=metrics.confusion_matrix(y_test, results)
sklearn_confusion=metrics.confusion_matrix(y_test, sk_predictions)
print('my confusion matrix:')
print(my_confusion)
print('sklearn confusin matrix:')
print(sklearn_confusion)

my confusion matrix:
[[22  2]
 [ 5 21]]
sklearn confusin matrix:
[[23  1]
 [ 2 24]]


### The Sklearn model performs better. 
- The better performace could be due the 'min_samples_leaf', 'splitter', or 'min_samples_spilt' parameters being different than min. 

### major plagiarism from:
https://github.com/random-forests/tutorials/blob/master/decision_tree.ipynb